In [1]:
!sudo apt-get update
!apt install caffe-cpu

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.cm as cm
from PIL import Image
import os
import scipy.io as sio
import time
# Make sure that caffe is on the python path:
#caffe_root = ''  # this file is expected to be in {caffe_root}/examples/hed/
import sys

#sys.path.insert(0, caffe_root + 'python')

import caffe

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://deve

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:


data_root = '/content/drive/MyDrive/2020Fall/capstone/testingdata/'
#load test data list
with open(data_root + 'test.txt') as f:
    test_lst = f.readlines
test_f = open(data_root + 'test.txt', 'r')
# test image list
im_test_list = []
#the corresponding mask image list
#mask_test_list = []
for im_pairs in test_f:
    im_test_list.append(im_pairs.split(' ')[0])
    #mask_test_list.append(im_pairs.split(' ')[1])

im_test_name_list=im_test_list
im_test_list = [data_root + x.strip() for x in im_test_list]
#mask_test_list = [data_root + x.strip() for x in mask_test_list]

print('!!!!!!!!!')
im_lst = []
#load image and substract mean value
for i in range(0, len(im_test_list)):
    im = Image.open(im_test_list[i])
    im = np.array(im, dtype=np.float32)
    im = im[:, :, ::-1]
    im -= np.array((127.00699, 126.66877, 126.67892))
    im_lst.append(im)
    print('finished,', i)

# Visualization of detection result
def plot_single_scale(scale_lst, size):
    pylab.rcParams['figure.figsize'] = size, size / 2

    plt.figure()
    for i in range(0, len(scale_lst)):
        s = plt.subplot(1, 5, i + 1)
        plt.imshow(1 - scale_lst[i], cmap=cm.Greys_r)
        image_name = 'output_' + str(i) + '.png'
        s.set_xticklabels([])
        s.set_yticklabels([])
        s.yaxis.set_ticks_position('none')
        s.xaxis.set_ticks_position('none')
    plt.tight_layout()


# Save detection results
def save_single_scale(scale_lst, name, outputpath):
    for i in range(0, len(scale_lst)):
        image_name = name  + '.png'
        plt.imsave(outputpath + image_name, 1 - scale_lst[i], cmap='gray')
        sio.savemat(outputpath + name, {'predmap': 1 - scale_lst[i]})


# set GPU model and ID
#caffe.set_mode_gpu()
#caffe.set_device(0)
print('begin')
model_root = '/content/drive/MyDrive/2020Fall/capstone/'
testnet='/content/drive/MyDrive/2020Fall/capstone/test_fphb_crack.prototxt'
valresults='/content/drive/MyDrive/2020Fall/capstone/'
modelList=['fphb_crack']#['train_val_fuse_fpn_crack_c1_v2_iter_4000']
#modelList=['train_val_fuse_fpn_crack_c1_fast_v2_iter_4000','train_val_fuse_fpn_crack_c1_fast_v2_iter_8000','train_val_fuse_fpn_crack_c1_fast_v2_iter_12000','train_val_fuse_fpn_crack_c1_fast_v2_iter_16000','train_val_fuse_fpn_crack_c1_fast_v2_iter_20000','train_val_fuse_fpn_crack_c1_fast_v2_iter_24000','train_val_fuse_fpn_crack_c1_fast_v2_iter_28000','train_val_fuse_fpn_crack_c1_fast_v2_iter_32000','train_val_fuse_fpn_crack_c1_fast_v2_iter_36000','train_val_fuse_fpn_crack_c1_fast_v2_iter_40000']
for model in modelList:
    #load network architecture and trained model
    net = caffe.Net(testnet, model_root +model+'.caffemodel', caffe.TEST)
    idx = 0
    outputpath=valresults+model+'-2'+'/'
    time_avg=0    
    if not os.path.exists(outputpath):
        os.makedirs(outputpath)
    for i,im in enumerate(im_lst):
        im = im.transpose((2, 0, 1))
        net.blobs['data'].reshape(1, *im.shape)
        net.blobs['data'].data[...] = im
        # run net and take argmax for prediction
        start = time.time()
        net.forward()
        end = time.time()
        time_per_image=end-start
        print('time each image',time_per_image)
        time_avg=time_avg+time_per_image
        print('time per image is %f' %time_per_image)
        out1 = net.blobs['sigmoid-dsn1'].data[0][0, :, :]
        out2 = net.blobs['sigmoid-dsn2'].data[0][0, :, :]
        out3 = net.blobs['sigmoid-dsn3'].data[0][0, :, :]
        out4 = net.blobs['sigmoid-dsn4'].data[0][0, :, :]
        out5 = net.blobs['sigmoid-dsn5'].data[0][0, :, :]
        fuse = net.blobs['sigmoid-fuse'].data[0][0, :, :]
        scale_lst=[fuse]
        print (i)
        #get image name
        print(im_test_name_list[0])
        imgname=im_test_name_list[i].split('.')[0]
        print (imgname)
        #plot_single_scale(scale_lst, 22)
        save_single_scale(scale_lst, imgname, outputpath)
        print('finished!!!!')

    print('average time is %f' %(time_avg/len(im_lst)))

!!!!!!!!!
finished, 0
finished, 1
finished, 2
finished, 3
finished, 4
finished, 5
finished, 6
finished, 7
finished, 8
finished, 9
finished, 10
finished, 11
finished, 12
finished, 13
finished, 14
finished, 15
finished, 16
finished, 17
finished, 18
finished, 19
finished, 20
finished, 21
finished, 22
finished, 23
finished, 24
finished, 25
finished, 26
finished, 27
finished, 28
finished, 29
finished, 30
finished, 31
finished, 32
finished, 33
finished, 34
finished, 35
finished, 36
finished, 37
finished, 38
finished, 39
finished, 40
finished, 41
finished, 42
finished, 43
finished, 44
finished, 45
finished, 46
finished, 47
finished, 48
finished, 49
finished, 50
finished, 51
finished, 52
finished, 53
finished, 54
finished, 55
finished, 56
finished, 57
finished, 58
finished, 59
finished, 60
finished, 61
finished, 62
finished, 63
finished, 64
finished, 65
finished, 66
finished, 67
finished, 68
finished, 69
finished, 70
finished, 71
finished, 72
finished, 73
finished, 74
finished, 75
finished, 76